# MEIRL

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
area = "london"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [3]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [4]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [5]:
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [6]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [10]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [11]:
def update_position(current_position, eps, reward):
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        updated_position = random.choice(value_list[value_list["value"] == max(value_list["value"])].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]
    else:
        updated_position = random.choice(value_list["value"].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]

    return updated_position

In [12]:
# how many iterations?
Y = []
iteration = 50
for i in range (iteration):
    epsilon = 1 / (i+1)  
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        start_position = random.choice(adjacency_matrix.columns)
        current_position = start_position
        # agent will walk for 50 steps
        for k in range(50):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = gradient + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 223.712587413
-0.0111944273284
1 145.0
0.191301631226
2 113.193667444
0.232912338082
3 104.443438914
0.30870402628
4 102.762779553
0.304553291695
5 107.080524058
0.326742990061
6 110.60380326
0.200551159888
7 104.234702797
0.267443469317
8 106.218269231
0.271929418178
9 113.875670841
0.251541117502
10 112.989870212
0.221669257671
11 116.254208754
0.188771908311
12 113.143884892
0.268605839635
13 111.047445797
0.306119000859
14 114.890511927
0.196225847562
15 114.000072569
0.292964446755
16 116.000600962
0.203972810063
17 125.594957674
0.226207652822
18 117.544039294
0.297689955122
19 126.278708134
0.19367441016
20 119.732172334
0.261964152123
21 117.883204633
0.226961563131
22 116.611286255
0.233882179249
23 137.903492213
0.26659442732
24 115.544384323
0.204673515651
25 117.693162393
0.176337836117
26 113.201743908
0.245592630261
27 109.15
0.232203320173
28 116.365384615
0.261393381376
29 115.554728426
0.220166486453
30 121.178238428
0.187416864058
31 134.783333333
0.172658388337
32 

# Local Depth

In [9]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [10]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [11]:
correlation_local_depth

0.45406804231592879

# Connectivity 

In [12]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.32460389310488996

# Global Depth

In [13]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.45406704994670616